# Training

In this notebook, the selected models are trained on the datasets generated by the coreset selectors.

### Models

- Resnet 18
- Mobilenet v2
- VGGnet 11
- Densenet 121

In [ ]:
from training_utilities import train_and_save_model

#### Train on "full" dataset: 20000 datapoints

In [ ]:
models = ['resnet', 'mobilenet', 'densenet', 'vgg']

for model in models:
    train_and_save_model(model_name=model,
                         coreset_percentage=1.0,
                         trainset_size='subset',
                         device='cuda:1'
                        )

#### Train for core-sets

In [ ]:
selectors = ['glister', 'k-centers', 'random']
percentages = [0.5, 0.3, 0.1]

for selector in selectors:
    for percentage in percentages:

        train_and_save_model(model_name='resnet',
                             coreset_selector=selector,
                             coreset_percentage=percentage,
                             trainset_size='subset',
                             device='cuda:1'
                            )

In [ ]:
selectors = ['glister', 'k-centers', 'random']
percentages = [0.5, 0.3, 0.1]

for selector in selectors:
    for percentage in percentages:

        train_and_save_model(model_name='mobilenet',
                             coreset_selector=selector,
                             coreset_percentage=percentage,
                             trainset_size='subset',
                             device='cuda:1'
                            )

In [ ]:
selectors = ['glister', 'k-centers', 'random']
percentages = [0.5, 0.3, 0.1]

for selector in selectors:
    for percentage in percentages:

        train_and_save_model(model_name='densenet',
                             coreset_selector=selector,
                             coreset_percentage=percentage,
                             trainset_size='subset',
                             device='cuda:1'
                            )

In [ ]:
selectors = ['glister', 'k-centers', 'random']
percentages = [0.5, 0.3, 0.1]

for selector in selectors:
    for percentage in percentages:

        train_and_save_model(model_name='vgg',
                             coreset_selector=selector,
                             coreset_percentage=percentage,
                             trainset_size='subset',
                             device='cuda:1'
                            )